In [1]:
import sqlite3
import pandas as pd
import jieba
# Create your connection.
cnx = sqlite3.connect('Data.db')

times = 0
df = pd.read_sql_query("SELECT * FROM rating", cnx)
df = pd.read_sql_query("SELECT * FROM rating WHERE star == 5", cnx)
comment_ = df['comment'].to_string()

stopwords = ['假貨','瑕疵','仿','低仿','高仿','糟糕','不好','差評','f','F','不太','不佳']
words = jieba.lcut(comment_)

for x in range (len(df['comment'])):
    comment_ = df['comment'][x]
    words = jieba.lcut(comment_)
#     words = jieba.cut_for_search(df['comment'][x])  # 搜索引擎模式
#     jieba.set_dictionary('./dict.txt.big.txt')
    for word in words:
        if word in stopwords:
            times+=1
    
print(times)

print(times/len(df['comment']),"%")
# print(df)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.874 seconds.
Prefix dict has been built succesfully.


505
0.04972430090586845 %


In [ ]:
print(df['comment'][0])

In [ ]:
import jieba
seg_list = jieba.cut(df['comment'][0], cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))

In [ ]:
all_comment = []
for x in range(len(df['comment'])):
    seg_list = jieba.cut_for_search(df['comment'][x])  # 搜索引擎模式
    jieba.set_dictionary('./dict.txt.big.txt')
    all_comment.append(",".join(seg_list))
# print(", ".join(seg_list))
print(all_comment)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(all_comment)

# print("查看對應表")
# print(vectorizer.vectorizer.vocabulary_)

print(vectorizer.get_feature_names())
x_ = vectorizer.get_feature_names()
print(type(x_))
print("\n查看IDF對應的分數")
# print(vectorizer.idf_)


# 編碼
vector = vectorizer.transform(all_comment)

print(vector.shape)

print("\n查看編碼後結果")
print(vector.toarray())

In [ ]:
len(x_)

In [ ]:
conn = sqlite3.connect('Data_word.db')
c = conn.cursor()
c.execute('''CREATE TABLE word
            (number int, context text)''') # REAL_0 FAKE_1
conn.commit()
# conn.close()

In [ ]:
for i in range(len(x_)):
    temp_list = [i+1,x_[i]]
    c.execute('INSERT INTO word VALUES (?,?)', temp_list)
    print(x_[i])

In [ ]:
conn.commit()
conn.close()